<p>
  <a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-sagemaker/blob/main/Lab%204%20-%20Exploring%20Data/exploring.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

First off, you'll also need to install a few packages.

In [1]:
!pip install --quiet --upgrade neo4j
!pip install --quiet google-cloud-storage

     |████████████████████████████████| 89 kB 4.3 MB/s 


You'll need to enter the credentials from your Neo4j instance below.

The default DB_NAME is always neo4j.

In [2]:
DB_URL = "neo4j://ec2-44-202-197-32.compute-1.amazonaws.com:7687"
DB_USER = "neo4j"
DB_PASS = "foo123"
DB_NAME = "neo4j"

In [3]:
import pandas as pd
from neo4j import GraphDatabase

driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASS))

In [7]:
# node labels
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        CALL db.labels() YIELD label
        CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as freq', {})
        YIELD value
        RETURN label, value.freq AS freq
      """
    ).data()
  )
df = pd.DataFrame(result)
display(df)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 6))



ClientError: ignored

In [ ]:
# relationship types
with driver.session(database=DB_NAME) as session:
    result = session.read_transaction(
        lambda tx: tx.run(
            """
      CALL db.relationshipTypes() YIELD relationshipType as type
      CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as freq', {})
      YIELD value
      RETURN type AS relationshipType, value.freq AS freq
      ORDER by freq DESC
      """
        ).data()
    )
df = pd.DataFrame(result)
display(df)

In [ ]:
# transaction types
with driver.session(database=DB_NAME) as session:
    result = session.read_transaction(
        lambda tx: tx.run(
            """
    MATCH (t:Transaction)
    WITH sum(t.amount) AS globalSum, count(t) AS globalCnt
    WITH *, 10^3 AS scaleFactor
    UNWIND ['CashIn', 'CashOut', 'Payment', 'Debit', 'Transfer'] AS txType
      CALL apoc.cypher.run('MATCH (t:' + txType + ')
        RETURN sum(t.amount) as txAmount, count(t) AS txCnt', {})
      YIELD value
    RETURN txType,value.txAmount AS TotalMarketValue
    """
        ).data()
    )
df = pd.DataFrame(result)
display(df)